In [1]:
import numpy as np
from numpy import linalg as LA
from scipy.io import loadmat
from scipy import optimize
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv
from tqdm import trange
import time
from get_dataset_info import *
from scipy.spatial.transform import Rotation
import sys

%matplotlib qt
%config InlineBackend.figure_format = 'retina'
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)


def plot_cameras_and_axes(ax, C_list, axis_list, s, valid_idx, col):

    for i in range(np.size(C_list,1)):

        C = C_list[:,i]
        axis = axis_list[:,i]
        ax.plot(C[0], C[1], C[2], 'o', color=col[i],  label='Camera {}'.format(valid_idx[i]+1), alpha=0.7)

        x_axis = C[0] + s*axis[0]
        y_axis = C[1] + s*axis[1]
        z_axis = C[2] + s*axis[2]

        ax.plot([x_axis, C[0]], [y_axis, C[1]], [z_axis, C[2]], '-', color=col[i], lw=3, alpha=0.7)

def plot_cameras_and_3D_points(X_arr, C_arr, axis_arr, s, valid_idx, multi=False):
    
    fig = plt.figure(figsize=(12,8))
    ax = plt.axes(projection='3d')
    col = cm.rainbow(np.linspace(0, 1, np.size(C_arr,1)))

    if multi:
        for i in range(len(X_arr)):
            X = X_arr[i]
            ax.plot(X[0], X[1], X[2], '.', color=col[i], ms=0.8)
    else:
        ax.plot(X_arr[0], X_arr[1], X_arr[2], '.', color='magenta', ms=0.4)
    plot_cameras_and_axes(ax, C_arr, axis_arr, s, valid_idx, col)

    ax.set_xlabel(r'$X$')
    ax.set_ylabel(r'$Y$')
    ax.set_zlabel(r'$Z$')
    ax.set_aspect('equal')
    # ax.view_init(elev=-45, azim=-45, roll=180)
    fig.tight_layout()
    plt.legend(loc="lower right")
    plt.show()

def plot_3D_points(X):
    
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.plot(X[0], X[1], X[2], '.', ms=1, color='magenta', label='X')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.set_aspect('equal')
    ax.legend(loc="lower right")
    fig.tight_layout()
    plt.show()

In [2]:
def compute_rotation_averaging(imgs, init_pair, K, pixel_threshold, plot=False):
    print('\n\n\n### Computing rotation averaging ###\n')

    K_inv = LA.inv(K)
    n_imgs = imgs.shape[0]
    n_camera_pairs = n_imgs-1

    marg = 0.75
    min_its = 5000
    max_its = 10000
    scale_its = 4
    alpha = 0.99
    P1 = cv.get_canonical_camera()
    rel_cameras = [P1]

    x1_norm_RA = []
    x2_norm_RA = []
    inliers_RA = []

    for i in range(n_camera_pairs):    
        print('\nCamera pair:', i+1, '/', n_camera_pairs)

        img1 = imgs[i]
        img2 = imgs[i+1]
        x1, x2, _, _ = cv.compute_sift_points(img1, img2, marg, flann=True, verbose=True)
        x1_norm = cv.dehomogenize(K_inv @ x1)
        x2_norm = cv.dehomogenize(K_inv @ x2)
        x1_norm_RA.append(x1_norm)
        x2_norm_RA.append(x2_norm)

        E, inliers = cv.estimate_E_robust(K, x1_norm, x2_norm, min_its, max_its, scale_its, alpha, pixel_threshold, essential_matrix=True, homography=True, verbose=True)
        inliers_RA.append(inliers)

        x1_norm_inliers = x1_norm[:,inliers]
        x2_norm_inliers = x2_norm[:,inliers]

        P2_arr = cv.extract_P_from_E(E)
        X_arr = cv.compute_triangulated_X_from_extracted_P2_solutions(P1, P2_arr, x1_norm_inliers, x2_norm_inliers)
        P2, X = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr, verbose=True)
        rel_cameras.append(P2)

        if plot:
            percentile = 90
            feasable_pts = cv.compute_feasible_points(P1, P2, X, percentile)
            P_arr = np.array([P1, P2])
            C_arr, axis_arr = cv.compute_camera_center_and_normalized_principal_axis(P_arr, multi=True)
            plot_cameras_and_3D_points(X[:,feasable_pts], C_arr, axis_arr, s=1, valid_idx=[0,1], multi=False)

    rel_cameras = np.array(rel_cameras)
    rel_rots = rel_cameras[:,:,:-1]
    abs_rots = cv.compute_absolute_rotations(rel_rots, init_pair[0], verbose=True)
    
    return abs_rots, x1_norm_RA, x2_norm_RA, inliers_RA 

In [3]:
def compute_initial_3D_points(imgs, init_pair, K, pixel_threshold, plot=False):
    print('\n\n\n### Computing initial 3D-points ###\n')

    K_inv = LA.inv(K)
    min_its = 40000
    max_its = 40000
    scale_its = 3
    alpha = 0.99
    marg = 0.75

    x1_init, x2_init, des1_init, des2_init = cv.compute_sift_points(imgs[init_pair[0]], imgs[init_pair[1]], marg, flann=True, verbose=True)
    x1_init_norm = cv.dehomogenize(K_inv @ x1_init)
    x2_init_norm = cv.dehomogenize(K_inv @ x2_init)

    E, inliers = cv.estimate_E_robust(K, x1_init_norm, x2_init_norm, min_its, max_its, scale_its, alpha, pixel_threshold, essential_matrix=True, homography=True, verbose=True)

    x1_init_norm_inliers = x1_init_norm[:,inliers]
    x2_init_norm_inliers = x2_init_norm[:,inliers]
    des1_init_inliers = des1_init[inliers]
    des2_init_inliers = des2_init[inliers]

    P1 = cv.get_canonical_camera()
    P2_arr = cv.extract_P_from_E(E)
    X_arr = cv.compute_triangulated_X_from_extracted_P2_solutions(P1, P2_arr, x1_init_norm_inliers, x2_init_norm_inliers)
    P2, X_init_inliers = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr, verbose=True)

    percentile = 90
    feasible_pts = cv.compute_feasible_points(P1, P2, X_init_inliers, percentile)

    x1_init_norm_feasible_inliers = x1_init_norm_inliers[:,feasible_pts]
    x2_init_norm_feasible_inliers = x2_init_norm_inliers[:,feasible_pts]
    des1_init_feasible_inliers = des1_init_inliers[feasible_pts]
    des2_init_feasible_inliers = des2_init_inliers[feasible_pts]
    X_init_feasible_inliers = X_init_inliers[:,feasible_pts]
    X_init_idx = np.ones(X_init_feasible_inliers.shape[1], dtype=bool)

    if plot:
        plot_3D_points(X_init_feasible_inliers)
    
    return x1_init_norm_feasible_inliers, x2_init_norm_feasible_inliers, des1_init_feasible_inliers, des2_init_feasible_inliers, X_init_feasible_inliers, X_init_idx

In [4]:
def compute_translation_registration(K, imgs, init_pair, pixel_threshold, abs_rots, x1_init_norm_feasible_inliers, x2_init_norm_feasible_inliers, des1_init_feasible_inliers, X_init_feasible_inliers, X_init_idx):
    print('\n\n\n### Computing translation registration ###\n')

    K_inv = LA.inv(K)
    marg = 0.75
    min_its = 10000
    max_its = 20000
    scale_its = 1
    alpha = 0.99

    trans = []
    x_norm_TR = []
    X_idx_TR = []
    inliers_TR = []

    n_imgs = imgs.shape[0]
    valid_cameras = np.ones(n_imgs, dtype=bool)

    for i in range(n_imgs):
        print('\nImage:', i+1, '/', n_imgs)

        if (i != init_pair[1]) and (i != init_pair[0]):
            img2 = imgs[i]            
            _, x2, X_idx = cv.compute_sift_points_TR(x1_init_norm_feasible_inliers, des1_init_feasible_inliers, img2, marg, flann=True, verbose=True)
            x_norm = cv.dehomogenize(K_inv @ x2)
        elif i == init_pair[0]:
            x_norm = x1_init_norm_feasible_inliers
            X_idx = X_init_idx
        elif i == init_pair[1]:
            x_norm = x2_init_norm_feasible_inliers
            X_idx = X_init_idx

        X = X_init_feasible_inliers[:,X_idx]        
        R = abs_rots[i]

        T, inliers = cv.estimate_T_robust(K, R, X[:-1], x_norm, min_its, max_its, scale_its, alpha, pixel_threshold, verbose=True)
        
        if np.isnan(T[0]):
            valid_cameras[i] = False

        x_norm_TR.append(x_norm)
        X_idx_TR.append(X_idx)
        trans.append(T)
        inliers_TR.append(inliers)
        
    trans = np.array(trans)

    return trans, valid_cameras, x_norm_TR, X_idx_TR, inliers_TR

In [5]:
def refine_rotations_and_translations(trans, abs_rots, X_init_feasible_inliers, valid_cameras, X_idx_TR, x_norm_TR, inliers_TR):
    print('\n\n\n### Refining translations and rotations ###\n')

    def fun(params, n_valid_cams, xs_norm, X_init, X_idx_TR, inliers_TR, valid_cameras):

        trans = params[:n_valid_cams * 3].reshape((n_valid_cams, 3))
        q_arr = params[n_valid_cams * 3:].reshape((n_valid_cams, 4))
        rots = []

        for i in range(n_valid_cams):
            R = Rotation.from_quat(q_arr[i] / LA.norm(q_arr[i])).as_matrix()
            U, _, VT = LA.svd(R, full_matrices=False)
            R = U @ VT
            rots.append(R)

        xs_proj = []
        t = 0
        for i in range(len(valid_cameras)):

            if valid_cameras[i]:

                X_idx = X_idx_TR[i]
                inliers_T = inliers_TR[i]

                R = rots[t] 
                X = X_init[:,X_idx][:,inliers_T]
                T = trans[t]
                t += 1

                x_proj = cv.dehomogenize(R @ X + T[:,None])
                xs_proj.append(x_proj)
        xs_proj = np.concatenate(xs_proj, 1)

        return (xs_proj - xs_norm).ravel()


    xs_norm = []
    n_trans = trans.shape[0]

    for i in range(n_trans):
        if valid_cameras[i]:
            x_norm = x_norm_TR[i]
            inliers_T = inliers_TR[i]
            xs_norm.append(x_norm[:,inliers_T])
    xs_norm = np.concatenate(xs_norm, 1)

    n_valid_cams = np.sum(valid_cameras)

    q_arr = []
    for i in range(n_trans): # n_imgs
        if valid_cameras[i]:
            R = abs_rots[i]
            q = Rotation.from_matrix(R).as_quat()
            q_arr.append(q)
    q_arr = np.concatenate(q_arr, 0)

    x0 = np.concatenate((trans[valid_cameras].ravel(), q_arr), 0)
    res = optimize.least_squares(fun, x0, method='lm', args=(n_valid_cams, xs_norm, X_init_feasible_inliers[:-1], X_idx_TR, inliers_TR, valid_cameras))

    trans_opt_valid = res.x[:n_valid_cams * 3].reshape((n_valid_cams, 3))
    q_opt_valid = res.x[n_valid_cams * 3:].reshape((n_valid_cams, 4))

    abs_rots_opt_valid = []
    for i in range(n_valid_cams):
        R = Rotation.from_quat(q_opt_valid[i] / LA.norm(q_opt_valid[i])).as_matrix()
        U, _, VT = LA.svd(R, full_matrices=False)
        R = U @ VT
        abs_rots_opt_valid.append(R)
    abs_rots_opt_valid = np.array(abs_rots_opt_valid)

    trans_opt = []
    abs_rots_opt = []
    t = 0

    for i in range(n_trans): # n_imgs
        if valid_cameras[i]:
            abs_rots_opt.append(abs_rots_opt_valid[t])
            trans_opt.append(trans_opt_valid[t])
            t += 1
        else:
            abs_rots_opt.append(abs_rots[i])
            trans_opt.append(trans[i])
    
    return abs_rots_opt, trans_opt

In [6]:
def create_cameras(abs_rots, trans):
    cameras = []

    for i in range(len(trans)):
        R = abs_rots[i]
        T = trans[i]
        P = np.column_stack((R, T))
        cameras.append(P)
    cameras = np.array(cameras)
    
    return cameras

In [7]:
def triangulate_final_3D_reconstruction(imgs, K, pixel_threshold, cameras, valid_cameras, inliers_RA, x1_norm_RA, x2_norm_RA):
    print('\n\n\n### Triangulating final 3D-reconstruction ###\n')

    K_inv = LA.inv(K)
    marg = 0.75
    alpha = 0.99

    X_final = []
    valid_idx = []
    n_valid_cameras = np.sum(valid_cameras)

    for i in range(valid_cameras.shape[0]):
        if valid_cameras[i]:
            valid_idx.append(i)
    valid_idx = np.array(valid_idx)
    print('Valid camera indices:', valid_idx)

    for idx in range(n_valid_cameras-1):
        print('Camera pair:', idx+1, '/', n_valid_cameras-1)

        i = valid_idx[idx]
        ij = valid_idx[idx+1]
        
        P1 = cameras[i]
        P2 = cameras[ij]

        if i+1 < ij:
            img1 = imgs[i]
            img2 = imgs[ij]
            x1, x2, _, _ = cv.compute_sift_points(img1, img2, marg, flann=True, verbose=True)
            x1_norm = cv.dehomogenize(K_inv @ x1)
            x2_norm = cv.dehomogenize(K_inv @ x2)

            min_its = 0
            max_its = 10000
            scale_its = 1
            E, inliers = cv.estimate_E_robust(K, x1_norm, x2_norm, min_its, max_its, scale_its, alpha, pixel_threshold, essential_matrix=True, homography=True, verbose=True)
        else:
            inliers = inliers_RA[i]
            x1_norm = x1_norm_RA[i]
            x2_norm = x2_norm_RA[i]
        
        x1_norm_inliers = x1_norm[:,inliers]
        x2_norm_inliers = x2_norm[:,inliers]

        percentile = 90
        X_inliers = cv.triangulate_3D_point_DLT(P1, P2, x1_norm_inliers, x2_norm_inliers, verbose=False)
        feasible_pts = cv.compute_feasible_points(P1, P2, X_inliers, percentile, ransac=False)
        X_final.append(X_inliers[:,feasible_pts])

    C_arr, axis_arr = cv.compute_camera_center_and_normalized_principal_axis(cameras[valid_idx], multi=True)
    plot_cameras_and_3D_points(X_final, C_arr, axis_arr, s=0.5, valid_idx=valid_idx, multi=True)

In [16]:
print('\n\n\n### Initializing ###\n')

data_set = 3
K, img_names, init_pair, pixel_threshold = get_dataset_info(data_set)
K_inv = LA.inv(K)
imgs = cv.load_image(img_names, multi=True)
n_imgs = imgs.shape[0]
n_camera_pairs = n_imgs-1




### Initializing ###



In [17]:
abs_rots, x1_norm_RA, x2_norm_RA, inliers_RA = compute_rotation_averaging(imgs, init_pair, K, pixel_threshold, plot=True)




### Computing rotation averaging ###


Camera pair: 1 / 13
Number of matches: 14186
Number of good matches: 7055
Iteration: 1 T_E: 10000 T_H: 10000 epsilon_E: 0.0 epsilon_H: 0 No. inliers: 14 From: E 8-point alg.
Iteration: 1 T_E: 10000 T_H: 10000 epsilon_E: 0.05 epsilon_H: 0.01 No. inliers: 346 From: H 4-point alg.
Iteration: 3 T_E: 10000 T_H: 10000 epsilon_E: 0.1 epsilon_H: 0.03 No. inliers: 690 From: H 4-point alg.
Iteration: 4 T_E: 10000 T_H: 10000 epsilon_E: 0.13 epsilon_H: 0.05 No. inliers: 903 From: H 4-point alg.
Iteration: 11 T_E: 10000 T_H: 10000 epsilon_E: 0.27 epsilon_H: 0.05 No. inliers: 1881 From: E 8-point alg.
Iteration: 49 T_E: 2576.0 T_H: 10000 epsilon_E: 0.54 epsilon_H: 0.06 No. inliers: 3803 From: H 4-point alg.


c:\Users\erikn\skola\EEN020-Computer-Vision\project\computer_vision.py:634: RuntimeWarning: divide by zero encountered in double_scalars
  T = scale * np.ceil(np.log(1-alpha) / np.log(1-epsilon**s))


Bailout at iteration: 2576
No. valid coords for each camera pair: [   2 7604 3803 3803]
Argmax(P2_arr): 1

Camera pair: 2 / 13
Number of matches: 13780
Number of good matches: 6965
Iteration: 1 T_E: 10000 T_H: 10000 epsilon_E: 0.04 epsilon_H: 0 No. inliers: 247 From: E 8-point alg.
Iteration: 2 T_E: 10000 T_H: 10000 epsilon_E: 0.05 epsilon_H: 0.03 No. inliers: 350 From: H 4-point alg.
Iteration: 8 T_E: 10000 T_H: 10000 epsilon_E: 0.06 epsilon_H: 0.03 No. inliers: 451 From: E 8-point alg.
Iteration: 13 T_E: 10000 T_H: 10000 epsilon_E: 0.1 epsilon_H: 0.08 No. inliers: 674 From: H 4-point alg.
Iteration: 26 T_E: 10000 T_H: 10000 epsilon_E: 0.23 epsilon_H: 0.08 No. inliers: 1568 From: E 8-point alg.
Iteration: 39 T_E: 10000 T_H: 10000 epsilon_E: 0.29 epsilon_H: 0.13 No. inliers: 2004 From: H 4-point alg.
Iteration: 211 T_E: 7140.0 T_H: 10000 epsilon_E: 0.47 epsilon_H: 0.13 No. inliers: 3306 From: E 8-point alg.
Iteration: 676 T_E: 4924.0 T_H: 10000 epsilon_E: 0.5 epsilon_H: 0.13 No. inlier

In [18]:
x1_init_norm_feasible_inliers, x2_init_norm_feasible_inliers, des1_init_feasible_inliers, des2_init_feasible_inliers, X_init_feasible_inliers, X_init_idx = compute_initial_3D_points(imgs, init_pair, K, 3*pixel_threshold, plot=True)




### Computing initial 3D-points ###

Number of matches: 13996
Number of good matches: 2697
Iteration: 1 T_E: 40000 T_H: 40000 epsilon_E: 0.03 epsilon_H: 0 No. inliers: 68 From: E 8-point alg.
Iteration: 3 T_E: 40000 T_H: 40000 epsilon_E: 0.06 epsilon_H: 0.02 No. inliers: 159 From: H 4-point alg.
Iteration: 6 T_E: 40000 T_H: 40000 epsilon_E: 0.08 epsilon_H: 0.09 No. inliers: 217 From: H 4-point alg.
Iteration: 6 T_E: 40000 T_H: 40000 epsilon_E: 0.09 epsilon_H: 0.09 No. inliers: 232 From: H 4-point alg.
Iteration: 16 T_E: 40000 T_H: 40000 epsilon_E: 0.19 epsilon_H: 0.21 No. inliers: 509 From: H 4-point alg.
Iteration: 16 T_E: 40000 T_H: 40000 epsilon_E: 0.23 epsilon_H: 0.21 No. inliers: 624 From: H 4-point alg.
Iteration: 389 T_E: 40000 T_H: 40000 epsilon_E: 0.24 epsilon_H: 0.22 No. inliers: 638 From: H 4-point alg.
Iteration: 1100 T_E: 40000 T_H: 40000 epsilon_E: 0.3 epsilon_H: 0.22 No. inliers: 797 From: E 8-point alg.
Iteration: 1333 T_E: 40000 T_H: 40000 epsilon_E: 0.51 epsilon_H:

c:\Users\erikn\skola\EEN020-Computer-Vision\project\computer_vision.py:30: RuntimeWarning: divide by zero encountered in true_divide
  x_deh = x/x[-1]
c:\Users\erikn\skola\EEN020-Computer-Vision\project\computer_vision.py:30: RuntimeWarning: invalid value encountered in true_divide
  x_deh = x/x[-1]


Iteration: 39267 T_E: 40000 T_H: 40000 epsilon_E: 0.81 epsilon_H: 0.22 No. inliers: 2196 From: E 8-point alg.
Bailout at iteration: 40000
No. valid coords for each camera pair: [   0 4392 2196 2196]
Argmax(P2_arr): 1


In [19]:
trans, valid_cameras, x_norm_TR, X_idx_TR, inliers_TR = compute_translation_registration(K, imgs, init_pair, 3*pixel_threshold, abs_rots, x1_init_norm_feasible_inliers, x2_init_norm_feasible_inliers, des1_init_feasible_inliers, X_init_feasible_inliers, X_init_idx)




### Computing translation registration ###


Image: 1 / 14
Number of matches: 1976
Number of good matches: 847
Iteration: 1 T: 10000 epsilon: 0.64 No. inliers: 543
Iteration: 4 T: 10000 epsilon: 0.66 No. inliers: 561
Iteration: 7 T: 10000 epsilon: 0.67 No. inliers: 566
Iteration: 20 T: 10000 epsilon: 0.67 No. inliers: 570
Iteration: 40 T: 10000 epsilon: 0.7 No. inliers: 594
Iteration: 534 T: 10000 epsilon: 0.71 No. inliers: 599
Iteration: 871 T: 10000 epsilon: 0.71 No. inliers: 602
Iteration: 1017 T: 10000 epsilon: 0.71 No. inliers: 603
Iteration: 7538 T: 10000 epsilon: 0.71 No. inliers: 604
Bailout at iteration: 10000

Image: 2 / 14
Number of matches: 1976
Number of good matches: 1029
Iteration: 1 T: 10000 epsilon: 0.09 No. inliers: 94
Iteration: 2 T: 10000 epsilon: 0.65 No. inliers: 673
Iteration: 4 T: 10000 epsilon: 0.76 No. inliers: 781
Iteration: 9 T: 10000 epsilon: 0.78 No. inliers: 801
Iteration: 11 T: 10000 epsilon: 0.78 No. inliers: 806
Iteration: 13 T: 10000 epsilon: 0.8 N

c:\Users\erikn\skola\EEN020-Computer-Vision\project\computer_vision.py:634: RuntimeWarning: divide by zero encountered in log
  T = scale * np.ceil(np.log(1-alpha) / np.log(1-epsilon**s))


Bailout at iteration: 10000

Image: 6 / 14
Number of matches: 1976
Number of good matches: 1406
Iteration: 1 T: 10000 epsilon: 0.52 No. inliers: 735
Iteration: 2 T: 10000 epsilon: 0.52 No. inliers: 737
Iteration: 64 T: 10000 epsilon: 0.53 No. inliers: 742
Iteration: 85 T: 10000 epsilon: 0.53 No. inliers: 746
Iteration: 378 T: 10000 epsilon: 0.54 No. inliers: 753
Iteration: 3629 T: 10000 epsilon: 0.54 No. inliers: 757
Iteration: 4988 T: 10000 epsilon: 0.54 No. inliers: 758
Bailout at iteration: 10000

Image: 7 / 14
Number of matches: 1976
Number of good matches: 1445
Iteration: 1 T: 10000 epsilon: 0.27 No. inliers: 391
Iteration: 2 T: 10000 epsilon: 0.27 No. inliers: 396
Iteration: 6 T: 10000 epsilon: 0.43 No. inliers: 619
Iteration: 21 T: 10000 epsilon: 0.5 No. inliers: 723
Iteration: 53 T: 10000 epsilon: 0.5 No. inliers: 729
Iteration: 131 T: 10000 epsilon: 0.51 No. inliers: 741
Iteration: 434 T: 10000 epsilon: 0.51 No. inliers: 744
Bailout at iteration: 10000

Image: 8 / 14
Number of

In [20]:
abs_rots_opt, trans_opt = refine_rotations_and_translations(trans, abs_rots, X_init_feasible_inliers, valid_cameras, X_idx_TR, x_norm_TR, inliers_TR)




### Refining translations and rotations ###



In [21]:
cameras = create_cameras(abs_rots, trans)
cameras_opt = create_cameras(abs_rots_opt, trans_opt)

In [22]:
triangulate_final_3D_reconstruction(imgs, K, pixel_threshold, cameras, valid_cameras, inliers_RA, x1_norm_RA, x2_norm_RA)
triangulate_final_3D_reconstruction(imgs, K, pixel_threshold, cameras_opt, valid_cameras, inliers_RA, x1_norm_RA, x2_norm_RA)




### Triangulating final 3D-reconstruction ###

Valid camera indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
Camera pair: 1 / 13
Camera pair: 2 / 13
Camera pair: 3 / 13
Camera pair: 4 / 13
Camera pair: 5 / 13
Camera pair: 6 / 13
Camera pair: 7 / 13
Camera pair: 8 / 13
Camera pair: 9 / 13
Camera pair: 10 / 13
Camera pair: 11 / 13
Camera pair: 12 / 13
Camera pair: 13 / 13



### Triangulating final 3D-reconstruction ###

Valid camera indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]
Camera pair: 1 / 13
Camera pair: 2 / 13
Camera pair: 3 / 13
Camera pair: 4 / 13
Camera pair: 5 / 13
Camera pair: 6 / 13
Camera pair: 7 / 13
Camera pair: 8 / 13
Camera pair: 9 / 13
Camera pair: 10 / 13
Camera pair: 11 / 13
Camera pair: 12 / 13
Camera pair: 13 / 13
